In [1]:
import warnings
warnings.simplefilter("ignore")
import os
from keras.applications.resnet import ResNet50,preprocess_input
from keras.models import Model
from keras.utils import load_img, img_to_array
from keras.models import Sequential
from keras.layers import GlobalMaxPool2D
import joblib
from tqdm import tqdm
tqdm.pandas()

In [2]:
project_path = ""
data_path = os.path.join(project_path,"data")
imgs_folder = os.path.join(data_path,"images")
models_folder = os.path.join(project_path,"models")

In [3]:
resnet_model = ResNet50(weights="imagenet",include_top=False,input_shape=(256,256,3))
resnet_model.trainable = False

In [4]:
model = Sequential([resnet_model,GlobalMaxPool2D()])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 global_max_pooling2d (Globa  (None, 2048)             0         
 lMaxPooling2D)                                                  
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________


In [5]:
def get_resnet50_features(model,image,verbose=0):
    image = preprocess_input(image)
    features = model.predict(image, verbose=0)
    return features

In [6]:
train_img_subfolders_path = os.path.join(imgs_folder,"train")
test_img_subfolders_path = os.path.join(imgs_folder,"test")
train_img_subfolders = os.listdir(train_img_subfolders_path)
test_img_subfolders = os.listdir(test_img_subfolders_path)

In [7]:
img_feature_extract = {}

for subfolder in train_img_subfolders: 
    train_img_category_path = os.path.join(train_img_subfolders_path,subfolder)
    for img_name in tqdm(os.listdir(train_img_category_path)):
        img_path = os.path.join(train_img_category_path,img_name)
        image = load_img(img_path, target_size=(256, 256))
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        feature = get_resnet50_features(model=model,image=image,verbose=0)
        image_name = img_name.split('.')[0]
        img_feature_extract[image_name] = feature.flatten()/256   
    print(f"finished creating features for {subfolder}")

100%|██████████| 648/648 [01:29<00:00,  7.27it/s]


finished creating features for boys_apparel


100%|██████████| 700/700 [01:44<00:00,  6.68it/s]


finished creating features for boys_footwear


100%|██████████| 456/456 [01:13<00:00,  6.18it/s]


finished creating features for girls_apparel


100%|██████████| 658/658 [01:52<00:00,  5.82it/s]

finished creating features for girls_footwear


In [8]:
joblib.dump(img_feature_extract,"data/features/resnet50_img_feature_extracts")

['data/features/resnet50_img_feature_extracts']